In [5]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [6]:
pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=9f5540089915a03e4f7d3066d6a02f544d12ee1eff055f3e5a187aae0f110a5a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark import SparkContext, SparkConf

# Настройка Spark

conf = SparkConf().setAppName("Simple RDD Example").setMaster("local[*]")

sc = SparkContext(conf=conf)

# 1. Создание RDD из списка чисел

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

rdd = sc.parallelize(numbers)

# 2. Трансформации: Фильтрация чётных чисел

even_numbers_rdd = rdd.filter(lambda x: x % 2 == 0)

# 3. Действие: Подсчёт суммы чётных чисел

sum_even_numbers = even_numbers_rdd.sum()

# Вывод результата

print("Чётные числа:", even_numbers_rdd.collect())

print("Сумма чётных чисел:", sum_even_numbers)

a = 5 + 6

print(a)

# Остановка SparkContext

sc.stop()

Чётные числа: [2, 4, 6, 8, 10]
Сумма чётных чисел: 30
11


Пример создания Spark Session на Python:


In [ ]:
from pyspark.sql import SparkSession

# Создание объекта SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()

# Здесь можно выполнять операции с DataFrames
# Закрытие SparkSession
spark.stop()

Пример создания Spark Session с параметрами конфигурации:



In [ ]:
from pyspark.sql import SparkSession

# Создание Spark Session с различными параметрами конфигурации
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.cores", "4") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.checkpoint.dir", "/path/to/checkpoint/dir") \
    .config("spark.sql.warehouse.dir", "/path/to/warehouse/dir") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

# Пример использования Spark Session
df = spark.read.json("/path/to/json/file")
df.show()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/path/to/json/file.

### Workflow приложения Spark


Запуск автономного приложения и инициализация SparkContext



In [ ]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Spark Workflow Example").getOrCreate()

# Получение SparkContext
sc = spark.sparkContext

- Приложение, написанное на Python, Scala, Java или R, запускается автономно.
- В этом приложении создается объект SparkSession (который включает SparkContext), инициализирующий необходимые компоненты для работы Spark.
- Только при наличии SparkContext приложение называется драйвером (Driver)


Запрос ресурсов у менеджера кластеров





*   Драйвер запрашивает у кластерного менеджера (например, YARN, Mesos, Kubernetes или Spark Standalone) ресурсы для выполнения задачи.
* Драйвер сообщает кластерному менеджеру, сколько ресурсов ему требуется (например, количество исполнителей, объем памяти и процессорные ядра).



In [ ]:
spark-submit \
  --master yarn \
  --deploy-mode cluster \
  --num-executors 10 \
  --executor-memory 4g \
  --executor-cores 2 \
  path/to/your_script.py

Запуск исполнителей менеджером кластеров



- Кластерный менеджер распределяет ресурсы и запускает процессы исполнителей (executors) на рабочих узлах (worker nodes).
- Каждый исполнитель запускает отдельный процесс, который будет выполнять задачи, переданные драйвером.

Запуск кода Spark драйвером





*   Драйвер начинает выполнение кода Spark, который включает чтение данных, выполнение преобразований (transformations) и действий (actions).
* Драйвер разделяет задачи на этапы (stages) и назначает их исполнителям.



In [ ]:
# Чтение данных
df = spark.read.csv("path/to/input.csv", header=True, inferSchema=True)

# Выполнение преобразований
df_filtered = df.filter(df["age"] > 30)
df_grouped = df_filtered.groupBy("city").count()

Выполнение заданий исполнителями и отправка результатов драйверу



* Исполнители получают задания от драйвера и начинают их выполнение.
* Каждый исполнитель выполняет часть работы, такой как вычисления на данных и возвращает результаты драйверу.




In [ ]:
# Действие, инициирующее выполнение заданий
result = df_grouped.collect()

Остановка SparkContext и освобождение ресурсов



* После завершения выполнения всех задач, драйвер останавливает SparkContext.
* Исполнители закрываются и освобождают выделенные им ресурсы, возвращая их обратно в кластер.

In [ ]:
# Остановка SparkSession и SparkContext
spark.stop()

### RDD vs Dataset vs Dataframe

RDD (Resilient Distributed Dataset)

Схема данных:

- В RDD схема данных неявно определяется структурой объектов, хранящихся в RDD.
- RDD является типизированной коллекцией, где каждый элемент может быть объектом любого типа, но Spark не хранит информацию о структуре этих данных.

In [ ]:
from pyspark import SparkContext

sc = SparkContext("local", "RDD Example")
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]
rdd = sc.parallelize(data)

# Схема данных неявно определяется структурой кортежей
print(rdd.collect())  # [('Alice', 1), ('Bob', 2), ('Cathy', 3)]


Методы rdd

In [15]:
from pyspark.sql import SparkSession
from operator import add

spark = SparkSession.builder \
        .appName("MySparkApp") \
        .config("spark.master", "local[*]") \
        .getOrCreate()

sc = spark.sparkContext

data = [1,2,3,3,4,4,5,6,7,8,9,10]
data2 = [1,2,3,3,40]
rdd = sc.parallelize(data)
rdd2 = sc.parallelize(data2)

rdd_mapped = rdd.map(lambda x: x*2) # метод map к каждому элементу применяет функцию
rdd_filtered = rdd.filter(lambda x: x%2 == 0) # метод filter отбирает элементы по условию
rdd_flat_mapped = rdd.flatMap(lambda x: [x, x*2]) # добавляет удвоенный элемент после обычного
rdd_dist = rdd.distinct() # выводит уникальные элементы
rdd_union = rdd.union(rdd2) # соединение
print(rdd.count()) # длина rdd
print(rdd.first())
print(rdd2.take(2)) # берет 2 элемента слева [:2]
print("reduce", rdd2.reduce(add)) # складываем все элементы используя модуль operator
def print_square(x):
  square = x*x
  print(f"Square of {x} is {square}")

rdd2.foreach(print_square) # метод foreach работает только если мы работаем на кластере (а не на local)
# локально метод foreach не работает
spark.stop()

12
1
[1, 2]
reduce 49


DataFrame

Схема данных:

- В DataFrame схема данных явная и включает имена столбцов и типы данных.
- Схема может быть автоматически определена при создании DataFrame из источника данных (например, CSV, JSON) или явно задана при создании DataFrame.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()
data = [("Alice", 1), ("Bob", 2), ("Cathy", 3)]

# Явное определение схемы
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Value", IntegerType(), True)
])

# Создание DataFrame с явной схемой
df = spark.createDataFrame(data, schema)
df.printSchema()

# Автоматическое определение схемы при чтении данных из CSV
df_auto = spark.read.csv("/path/to/csv/file", header=True, inferSchema=True)
df_auto.printSchema()

Схема в DataFrame

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder \
.appName("schema") \
.config("spark.master", "local[*]") \
.getOrCreate()

data = [("Alice", 25), ("Artem", 26), ("Ivan", 48)]

# Схема - название и типы полей в таблице
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
])# последний параметр true - говорит может ли быть null значение (nullable = true)

df = spark.createDataFrame(data, schema)

df.printSchema()

df.show()

spark.stop()



root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|Artem| 26|
| Ivan| 48|
+-----+---+



Автоматическое создание схемы

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder \
.appName("schema") \
.config("spark.master", "local[*]") \
.getOrCreate()

data = [{"name":"Alice", "age":29},
        {"name":"Artem", "age":120},
        {"name":"Ivan", "age":56}]


df = spark.createDataFrame(data)

df.printSchema()

df.show()

spark.stop()

# тип age стал long - не очень подходит, но в принципе данный способ хорош

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

+---+-----+
|age| name|
+---+-----+
| 29|Alice|
|120|Artem|
| 56| Ivan|
+---+-----+



Создание датафрейма из RDD и методы работы с датафреймами

In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import avg, max

spark = SparkSession.builder \
.appName("schema") \
.config("spark.master", "local[*]") \
.getOrCreate()

data = [("Alice", 29), ("Bob", 87)]

rdd = spark.sparkContext.parallelize(data) # можно делать сразу

df = rdd.toDF(["name", "age"])# если не указать, то названия столбцов будут созданы автоматически

df.printSchema()

df.show(1) #truncate=False - показывает таблицу полностью

print(df.head(1)) # берет шапку - не выводит

print(df.take(3)) # берет шапку в виде списка - не выводит

columns = df.columns
print(columns) # returns list

columns = df.dtypes
print(columns) # returns list of tuples ('column', 'dtype')

# работают здесь и методы SQL
df.select("name").show()
df.selectExpr("name", "age+1 as age_plus_one").show()

df.filter(df["age"] > 30).show() # то же самое с методом where

df.groupBy("age").count().show()# группировка
df.groupBy("age").agg(
    avg("age").alias("average_age"),
    max("age").alias("max_age")
).show() # агрегация

df.orderBy("age").show() # df.sort("age", ascending=True)
df.orderBy(df["age"].desc()).show()# df.sort("age", ascending=False)

df.withColumn("age_plus_one", df["age"] + 1).show() # добавление столбца

df.withColumnRenamed("age", "fio").show() # переименовать столбец

df.drop("age_plus_one").show() # удаление столбца

df.distinct().show() # отобразить без строк-дубликатов

df.limit(1).show() # 1 первую строку

spark.stop()


root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)

+-----+---+
| name|age|
+-----+---+
|Alice| 29|
+-----+---+
only showing top 1 row

[Row(name='Alice', age=29)]
[Row(name='Alice', age=29), Row(name='Bob', age=87)]
['name', 'age']
[('name', 'string'), ('age', 'bigint')]
+-----+
| name|
+-----+
|Alice|
|  Bob|
+-----+

+-----+------------+
| name|age_plus_one|
+-----+------------+
|Alice|          30|
|  Bob|          88|
+-----+------------+

+----+---+
|name|age|
+----+---+
| Bob| 87|
+----+---+

+---+-----+
|age|count|
+---+-----+
| 29|    1|
| 87|    1|
+---+-----+

+---+-----------+-------+
|age|average_age|max_age|
+---+-----------+-------+
| 29|       29.0|     29|
| 87|       87.0|     87|
+---+-----------+-------+

+-----+---+
| name|age|
+-----+---+
|Alice| 29|
|  Bob| 87|
+-----+---+

+-----+---+
| name|age|
+-----+---+
|  Bob| 87|
|Alice| 29|
+-----+---+

+-----+---+------------+
| name|age|age_plus_one|
+-----+---+------------+
|Alice| 29|          

In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import avg, max

spark = SparkSession.builder \
.appName("schema") \
.config("spark.master", "local[*]") \
.getOrCreate()

df1 = spark.createDataFrame([(1, "artem"), (2, "bob"), (3, "ivan")], ["id", "name"])
df2 = spark.createDataFrame([(1, 100), (2, 200)], ["id", "money"])

df_join = df1.join(df2, "id", "left")
df_join.show()

df1.sample(withReplacement=False, fraction=0.5).show() #случайная выборка данных из датафрейма(50%)

+---+-----+-----+
| id| name|money|
+---+-----+-----+
|  1|artem|  100|
|  3| ivan| NULL|
|  2|  bob|  200|
+---+-----+-----+

+---+-----+
| id| name|
+---+-----+
|  1|artem|
+---+-----+



Dataset (доступен только в Scala и Java)


Схема данных:

- В Dataset схема данных явная и определяется структурой типизированных объектов (классов), хранящихся в Dataset.
- Dataset сочетает в себе преимущества RDD (типизированный API) и DataFrame (явная схема и оптимизация).

In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

val spark = SparkSession.builder.appName("Dataset Example").getOrCreate()
import spark.implicits._

case class Person(name: String, age: Int)
val data = Seq(Person("Alice", 1), Person("Bob", 2), Person("Cathy", 3))
val ds = data.toDS()

// Схема данных определяется структурой класса Person
ds.printSchema()